In [ ]:
import numpy as np
import random
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import pickle
import deepRD
from deepRD.diffusionIntegrators import langevinNoiseSampler, langevin
from deepRD.potentials import harmonic
from deepRD.noiseSampler import noiseSampler
from deepRD.noiseSampler import binnedData_qi, binnedData_ri
import deepRD.tools.trajectoryTools as trajectoryTools
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
localDataDirectory = './data/'
numSimulations = 100

In [ ]:
# Load binning sampling models
binnedDataFilename = localDataDirectory + 'qiBinnedData.pickle'
#binnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
binnedData = pickle.load(open(binnedDataFilename, "rb" ))
parameters = binnedData.parameterDictionary
print(parameters)

In [ ]:
# Define particle list
D = parameters['D']
position = [0,0,0]
velocity = [0,0,0]
mass =  parameters['mass']
particle1 = deepRD.particle(position, D, velocity, mass)
particleList = deepRD.particleList([particle1])

In [ ]:
# Define noise sampler
#nSampler = noiseSampler()
nSampler = noiseSampler(binnedData)

In [ ]:
# Define external potential
kconstant = 0.3
harmonicPotential = harmonic(kconstant)

In [ ]:
# Define integrator
#dt = binnedData.lagTimesteps * parameters['dt'] * parameters['stride'] #wrong
dt = parameters['dt']
integratorStride = 5
tfinal = 10000 #100000
KbT = parameters['KbT']
boxsize = parameters['boxsize']
#integType = "ABOBA"
equilibrationSteps = 2000
diffIntegrator = langevinNoiseSampler(dt, integratorStride, tfinal, nSampler, KbT, 
                                      boxsize, equilibrationSteps = equilibrationSteps)
diffIntegrator.setExternalPotential(harmonicPotential)

In [ ]:
particleIndex = 0
xpos = np.empty(0)
ypos = np.empty(0)
zpos = np.empty(0)
xvel = np.empty(0)
yvel = np.empty(0)
zvel = np.empty(0)
for i in range(numSimulations):
    print("Simulation ", i+1)
    position = position = np.array([boxsize*random.random()-0.5*boxsize,
                                    boxsize*random.random()-0.5*boxsize,
                                    boxsize*random.random()-0.5*boxsize])
    particle1 = deepRD.particle(position, D, velocity, mass)
    particleList = deepRD.particleList([particle1])
    t, X, V= diffIntegrator.propagate(particleList)
    xpos = np.concatenate((xpos, X[:,particleIndex,0]))
    ypos = np.concatenate((ypos, X[:,particleIndex,1]))
    zpos = np.concatenate((zpos, X[:,particleIndex,2]))
    xvel = np.concatenate((xvel, V[:,particleIndex,0]))
    yvel = np.concatenate((yvel, V[:,particleIndex,1]))
    zvel = np.concatenate((zvel, V[:,particleIndex,2]))
    print("")

In [ ]:
numbins=50
#averagepos = (np.array(xpos)+np.array(ypos)+np.array(zpos))/3.0
#plt.hist(averagepos, bins = numbins, density= True, alpha =0.3)
plt.hist(xpos, bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(ypos, bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(zpos, bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
#plt.xlim([-0.15,0.15])
plt.show()

In [ ]:
numbins=50
#averagepos = (np.array(xpos)+np.array(ypos)+np.array(zpos))/3.0
#plt.hist(averagepos, bins = numbins, density= True, #alpha =0.3)
plt.hist(xvel, bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(yvel, bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(zvel, bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
#plt.xlim([-0.15,0.15])
plt.show()

## Load one benchmark trajectory

In [ ]:
import deepRD.tools.trajectoryTools as trajectoryTools
import os

In [ ]:
parentDirectory = os.environ.get('MSMRD') + '/data/MoriZwanzig/benchmark_onetrajectory/'
fnamebase = parentDirectory + 'simMoriZwanzig_'

In [ ]:
# Load trajectory data from h5 files (only of distinguished particle)
trajs = []
print("Loading data ...")
for i in range(numSimulations):
    traj = trajectoryTools.loadTrajectory(fnamebase, i)
    trajs.append(traj)    
    print("File ", i+1, " of ", numSimulations, " done.", end="\r")

In [ ]:
# Plot distributions
xpos_ref = []
ypos_ref = []
zpos_ref = []
xvel_ref = []
yvel_ref = []
zvel_ref = []
numbins = 50
for traj in trajs:
    for i in range(len(traj)):
        xpos_ref.append(traj[i][1])
        ypos_ref.append(traj[i][2])
        zpos_ref.append(traj[i][3])
        xvel_ref.append(traj[i][4])
        yvel_ref.append(traj[i][5])
        zvel_ref.append(traj[i][6])
averagepos = (np.array(xpos_ref)+np.array(ypos_ref)+np.array(zpos_ref))/3.0
#plt.hist(averagepos, bins = numbins, density= True, alpha =0.3)
plt.hist(np.array(xpos_ref), bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(np.array(ypos_ref), bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(np.array(zpos_ref), bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
plt.show()

In [ ]:
plt.hist(np.array(xvel_ref), bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(np.array(yvel_ref), bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(np.array(zvel_ref), bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
plt.show()

## Comparison

In [ ]:
numbins=50
plt.hist(np.array(xvel), bins = numbins, density= True, alpha=0.5, label='reduced')
plt.hist(np.array(xvel_ref), bins = numbins, density= True, alpha=0.5, label='benchmark')
density = stats.kde.gaussian_kde(xvel)
#vv = np.arange(-1., 1, .02)
#plt.plot(vv, density(vv))
plt.legend()
plt.xlabel("velocity")
plt.ylabel("velocity distribution")
#plt.show()
plt.savefig("velocity_test.pdf", bbox_inches='tight')

In [ ]:
numbins=50
plt.hist(np.array(zpos), bins = numbins, density= True, alpha=0.5, label='reduced')
plt.hist(np.array(zpos_ref), bins = numbins, density= True, alpha=0.5, label='benchmark')
plt.legend()
plt.xlabel("position")
plt.ylabel("position distribution")
plt.xlim([-5,5])
plt.show()
#plt.savefig("position_test.pdf", bbox_inches='tight')

In [ ]:
# Functions to calculate moments
def calculateMean(trajs):
    meanPosition = np.zeros(3)
    totalSamples = 0
    for traj in trajs:
        for i in range(len(traj)):
            meanPosition += traj[i][1:4]
        totalSamples += len(traj)
    meanPosition = meanPosition/totalSamples
    return meanPosition

def calculateVariance(trajs, mean = None):
    if mean.any() == None:
        mean = calculateMean(trajs)
    variance = np.zeros(3)
    totalSamples = 0
    for traj in trajs:
        for i in range(len(traj)):
            devFromMean = traj[i][1:4] - mean
            variance += devFromMean*devFromMean
        totalSamples += len(traj)
    variance = variance/totalSamples
    return variance

def calculateStdDev(trajs, mean = None):
    variance = calculateVariance(trajs, mean)
    stddev = np.sqrt(variance)
    return stddev

def calculateAutoCorrelation(trajs, mean)